# **Reinforcement learning tutorial**
In today's tutorial you will learn how apply Q-learning algorithm and deep Q-networks on simulated environments.

We will use [**TensorFlow**](https://ekababisong.org/gcp-ml-seminar/tensorflow/) framework and [**Keras**](https://keras.io/) open-source library to rapidly prototype deep neural networks.

# **Preliminary operations**
The following code installs:
- some supplementary modules useful to visualization purposes;


In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install gym==0.19.0
!pip install pyglet==1.5.27

# **OpenAI Gym**
[**Gym**](https://www.gymlibrary.ml/) is a toolkit, developed by the [**OpenAI**](https://openai.com/) company, providing an easy to set up, general-intelligence benchmark with a wide variety of different environments for developing and comparing reinforcement learning algorithms.

# **Useful modules import**
First of all, it is necessary to import useful modules used during the tutorial.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import gym
import random
import time
import statistics
import cv2
import uuid
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output
from tensorflow import keras
from tensorflow.keras import layers
from collections import deque
from pyvirtualdisplay import Display

# **Utility functions**
Execute the following code to define some utility functions used in the tutorial:
- **print_taxi_step** draws a single frame of the taxi environment with the corresponding information;
- **print_taxi_single_episode** visualizes an entire episode of the taxi environment;
- **plot_training_rewards** draws in a graph the total reward trend and its moving average reached during the different episodes of the training process;
- **plot_stacked_frames** shows a stack of frames;
- **create_mp4_video_from_frames** creates an MP4 video file from a list of frames.

In [ ]:
def print_taxi_step(frame, episode, step, state, action, reward, current_total_reward):
  print(frame)
  if episode != None:
    print('Episode: ', episode)
  if step != None:
    print('Step: ', step)
  print('State: ', state)
  print('Action: ', action)
  print('Reward: ', reward)
  print('Total reward: ', current_total_reward)

def print_taxi_single_episode(frames, max_steps = 50, seconds_to_sleep = .1, episode = None):
  for i, frame in enumerate(frames):
    if i >= max_steps:
      break

    clear_output(wait = True)
    print_taxi_step(frame['frame'], episode, i, frame['state'], frame['action'], frame['reward'], frame['total_reward'])
    time.sleep(seconds_to_sleep)

def plot_training_rewards(rewards, moving_avg_window_size = None):
  if moving_avg_window_size is not None:
    moving_avg_total_reward = []
    for i in range(len(rewards)):
      window = rewards[max(0, i - moving_avg_window_size + 1) : i + 1]
      window_avg = statistics.mean(window)
      moving_avg_total_reward.append(window_avg)

  fig, ax = plt.subplots(figsize = (15, 5))

  ax.plot(range(len(rewards)), rewards,label = 'Total reward', color = 'orange')
  ax.set_xlabel('Episodes')
  if moving_avg_window_size is not None:
    ax.plot(range(len(moving_avg_total_reward)), moving_avg_total_reward, label = 'Total reward moving average')
    ax.legend(loc = 'upper left')

def plot_stacked_frames(stacked_frames):
  _, axs = plt.subplots(1, stacked_frames.shape[2], figsize = (15, 5))
  for i in range(stacked_frames.shape[2]):
    axs[i].axis('off')
    axs[i].imshow(stacked_frames[:, :, i],cmap = 'gray')

def create_mp4_video_from_frames(frames,fps):
  temp_video_path = 'tempfile.mp4'
  compressed_path = '{}.mp4'.format(str(uuid.uuid4()))
  
  size=(frames[0].shape[1], frames[0].shape[0])
  out = cv2.VideoWriter(temp_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
  
  for i in range(len(frames)):
      out.write(frames[i][..., ::-1].copy())  #rgb[...,::-1].copy()
  out.release()

  os.system(f"ffmpeg -i {temp_video_path} -vcodec libx264 {compressed_path}")

  os.remove(temp_video_path)

  return compressed_path

# **Useful terms**

- **Agent**: an entity that can perceive/explore the environment and act upon it.

- **Environment**: where the agent learns and decides what actions to perform.

- **Action**: actions are the moves taken by an agent within the environment.

- **State**: state is a situation returned by the environment after each action taken by the
agent.

- **Reward**: a scalar feedback returned to the agent from the environment when it performs specific actions.

- **Policy**: policy is a strategy applied by the agent for the next action based on the current state. It defines the agent behavior at a given time by mapping state to action.

<h1><center><img src=https://raw.githubusercontent.com/nderus/Generative-models-Lesson/main/images/pacman.png width="400"></center></h1>

# **Reinforcement Learning**
- Reinforcement learning (RL) is a subfield of machine learning that deals with the task of training an agent to make sequential decisions based on the rewards or penalties it receives for each action taken.

- In RL, an agent interacts with an environment and learns to take actions that maximize a cumulative reward signal over time.

- RL has gained a lot of attention in recent years due to its success in various applications, such as game playing, robotics, and medical decision-making.

- The core idea behind RL is to use trial-and-error learning to discover the optimal strategy for the agent. The agent receives feedback in the form of rewards or penalties for each action taken, and based on this feedback, it learns to adjust its behavior to maximize the overall reward. 

# **Q-learning**
- Q-learning is a popular reinforcement learning algorithm that is used to solve Markov decision processes (MDPs).

- The algorithm works by iteratively updating estimates of the optimal action-value function Q(s,a), which represents the expected reward of taking action a in state s and following the optimal policy thereafter.

- At each iteration, the algorithm selects an action based on an epsilon-greedy policy, which balances exploration and exploitation, and observes the reward and the next state. It then uses the observed information to update the estimate of Q(s,a) using the Bellman equation:

$$Q(s,a) ← Q(s,a) + α [r + γ max_a' Q(s',a') - Q(s,a)]$$

where α is the learning rate, r is the observed reward, γ is the discount factor, s' is the next state, and a' is the action that maximizes the expected reward from s'.

- The algorithm repeats this process until the estimates of Q converge to their optimal values, which is guaranteed to happen under certain assumptions. Once the optimal Q-function has been learned, the optimal policy can be derived by selecting the action with the highest Q-value in each state.

- Q-learning is a powerful and versatile algorithm that has been applied to a wide range of problems, from playing games to controlling robots. However, it can be slow to converge in large and complex environments, and its performance can be sensitive to the choice of hyperparameters.

# **Q-learning with self-driving taxi**
In this section, the Q-Learning algorithm is applied to a self-driving taxi that will need to learn how to transport its passengers to the desired destination. 

To this purpose, the [**Taxi**](https://www.gymlibrary.ml/environments/toy_text/taxi/) environment provided by Gym will be used:

"*There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.*"

## **Taxi environment**
The environment is represented by a training area for the self-driving taxi where to learn transport people in four different locations (R, G, Y, B):

<img src=https://raw.githubusercontent.com/nderus/Generative-models-Lesson/main/images/Reinforcement_Learning_Taxi_Env.png width="400">

### **Creation**
The following code creates the Taxi environment.

In [ ]:
env = gym.make('Taxi-v3')

### **Visualization**
To visualize the environment, the [**render**](https://www.gymlibrary.ml/content/api/#rendering) method can be used:
- the filled square represents the taxi, which is yellow without a passenger and green with a passenger;
- the pipe ("|") represents a wall which the taxi cannot cross;
- R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

In [ ]:
env.render()

### **State space**
Assuming self-driving taxi is the only vehicle in this parking lot, the parking lot can be divided into a $5\times 5$ grid, obtaining 25 possible taxi locations. Four of them are locations where the taxi can pick up and drop off a passenger: R, G, Y, B or $[(0,0), (0,4), (4,0), (4,3)]$ in $(row, col)$ coordinates.

Moreover, there are 4 possible destinations and 5 (R, G, Y, B or inside the taxi) passenger locations.

The taxi environment has $5\times 5\times 5\times 4=500$ total possible states.

Every Gym environment comes with an [**observation_space**](https://www.gymlibrary.ml/content/api/#attributes) attribute describing the format of valid observations (or states). If the space is composed by a fixed number of discrete values, an instance of the [**Discrete**](https://www.gymlibrary.ml/content/api/#spaces) space class is returned and the **n** property can be used to get the cardinality of the space.

In [ ]:
print('There are {} possible states'.format(env.observation_space.n))

### **Action space**
The action space is composed by six possible actions:
- south;
- north;
- east;
- west;
- pickup;
- dropoff.

Every Gym environment comes with an [**action_space**](https://www.gymlibrary.ml/content/api/#additional-environment-api) attribute describing the format of valid actions. As for the state space, the action space is discrete and the number of possible actions can be retrieved by the **n** property.

In [ ]:
print('There are {} possible actions'.format(env.action_space.n))

### **Rewards**
The rewards are:
- -1 for each step;
- -1 for every wall hit (the taxi will not move anywhere);
- +20 for successfully deliver the passenger;
- -10 for illegal actions (pick up or put down the passenger in the wrong location).

## **Exercise 1: Q-learning algorithm**
Implement the **q_learning** function given:
- the environment (*env*);
- the number of episodes (*episode_count*);
- the maximum number of steps per episode (*episode_max_steps*);
- the maximum value of $\epsilon$ (*max_epsilon*);
- the minimum value of $\epsilon$ (*min_epsilon*);
- the $\epsilon$ decay value (*epsilon_decay*);
- the learning rate $\alpha$ (alpha);
- the discount factor $\gamma$ (gamma).

It returns the learned Q-table (*Q*) and a **List** (*learning_history*) containing useful information to visualize the agent progresses during the learning process.

The following image shows the pseudo-code of the *Q-learning* algorithm.

<img src=https://raw.githubusercontent.com/nderus/Generative-models-Lesson/main/images/Q_learning_algorithm.png width="800">

The following parts need to be implemented:
1. initialize *Q* as a Numpy zero matrix with as many rows as the states and as many columns as the actions. The [**zeros**](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) function provided by the Numpy module can be used;
2. initialize $\epsilon$ equal to *max_epsilon*;
3. call the **reset** method provided by the Gym environment interface to reset the environment and get the random initial state ($s_0$);
4. decide whether to pick a random action or to exploit the already computed Q-values. To this purpose, the **uniform** function provided by the [**random**](https://docs.python.org/3/library/random.html) module can be used to generate a random number in the range $[0;1]$ to be compared against $\epsilon$. If a random action needs to be selected from the set of all possible actions, the **action_space.sample** method provided by the Gym environment can be exploited. Otherwise, use the [**argmax**](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) function provided by Numpy to select the optimal action for state $s_t$;
5. call the **step** method provided by the Gym environment interface to
execute the action $a_t$ selected at the previous point. It returns the new observation (or state) $s_{t+1}$, the reward $r_{t+1}$ achieved by action $a_t$, a boolean flag indicating if the episode has terminated (*True*) or not (*False*) and other additional information;
6. update $Q(s_t,a_t)$ using the *Bellman* equation. The Numpy [**amax**](https://numpy.org/doc/stable/reference/generated/numpy.amax.html) function can be used to find the maximum value of $Q(s_{t+1},a)$;
7. reduce $\epsilon$. It is progressively reduced during the learning process because, while the agent learns more and more about the environment, less and less exploration will be needed;

In [ ]:
def q_learning(env,episode_count,episode_max_steps,max_epsilon,min_epsilon,epsilon_decay,alpha,gamma):
  # 1. initialize Q as a zero matrix with as many rows as the states and as many columns as the actions 
  Q = np.zeros((env.observation_space.n, env.action_space.n))

  learning_history = [] # for visualization purposes

  # 2. inizialize epsilon equal to max_epsilon
  epsilon=max_epsilon

  for episode in range(episode_count):
      # for visualization purposes
      episode_history = [] 
      episode_total_reward=0
      
      # 3. reset the environment
      state = env.reset()
      
      step_count=0  #t
      done=False
      while (step_count<episode_max_steps) and (not done):
          # 4. decide whether to pick a random action or to exploit the already computed Q-values
          if(random.uniform(0,1) <= epsilon):
            action = env.action_space.sample()  #--> exploration
          else:
            action = np.argmax(Q[state])  #--> exploitation
    
          # 5. take the action and observe the outcome state and reward
          new_state, reward, done, _ = env.step(action)

          # 6. update Q(s,a)
          Q[state][action] = (1 - alpha) * Q[state][action] + alpha * (reward + gamma * np.amax(Q[new_state]))

          # put the current frame into a list for visualization purposes
          episode_total_reward+=reward
          episode_history.append(
                                  {
                                  'frame': env.render(mode='ansi'),
                                  'state': new_state,
                                  'action': action,
                                  'reward': reward,
                                  'total_reward' : episode_total_reward,
                                  }
                                )

          # update the current state
          state = new_state

          # increase the step count
          step_count+=1

      # 7. reduce epsilon
      if (epsilon>min_epsilon):
        epsilon=max(min_epsilon,epsilon-epsilon_decay)

      # put all the episode frames into a list for visualization purposes
      learning_history.append(episode_history)

  # return Q-table and history
  return Q, learning_history

## **Learning**
Now we are ready to start the learning process by calling the **q_learning** function.

The following parameters must be set before being passed to the function:
- the number of episodes (*episode_count*);
- the maximum number of steps per episode (*episode_max_steps*);
- the maximum value of $\epsilon$ (*max_epsilon*);
- the minimum value of $\epsilon$ (*min_epsilon*);
- the $\epsilon$ decay value (*epsilon_decay*);
- the learning rate $\alpha$ (alpha);
- the discount factor $\gamma$ (gamma).

In [ ]:
episode_count = 2000          # Total number of training episodes
episode_max_steps = 200       # Maximum number of steps per episode

max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001           # Minimum exploration probability 
epsilon_decay = 0.01          # Decay for exploration probability

alpha = 0.5                   # Learning rate
gamma = 0.99                  # Discount factor

start_time = time.time()
Q,learning_history = q_learning(env, episode_count, episode_max_steps, max_epsilon, min_epsilon, epsilon_decay, alpha, gamma)
print('Learning time: {:.1f}s'.format(time.time() - start_time))

### **Visualize the learning process**
It is important to observe how performance changes over time during the learning process.

The following code draws in a graph the number of steps and the total reward reached during the different episodes of the learning process.

In [ ]:
steps=[]
total_rewards=[]
for episode in learning_history:
  steps.append(len(episode))
  total_rewards.append(episode[-1]['total_reward'])

fig, ax1 = plt.subplots(figsize=(15, 5))

ax1.plot(range(len(steps)), steps, color = 'orange')
ax1.tick_params(axis = 'y', labelcolor = 'orange')
ax1.set_xlabel('Episodes')
ax1.set_ylabel('Steps', color = 'orange')

ax2 = ax1.twinx()
ax2.plot(range(len(total_rewards)), total_rewards, color = '#1f77b4')
ax2.tick_params(axis = 'y', labelcolor = '#1f77b4')
ax2.set_ylabel('Total reward', color = '#1f77b4')

plt.show()

To better highlight how the self-driving taxi improves during the learning process, it is useful to visualize single episodes using the **print_taxi_single_episode** function defined above. The following code shows the first episode of the learning process (only the first 25 steps per episode are displayed).

In [ ]:
print_taxi_single_episode(learning_history[0], 25, 0.5, 0)

While initially the taxi randomly moves exploring the environment, at the end of the learning process the agent is perfectly capable to pickup a passenger and to transport him to the desired destination. The following code shows the last episode of the learning process.

In [ ]:
print_taxi_single_episode(learning_history[-1], 25, 0.5, len(learning_history) - 1)

## **Performance evaluation**
Now it is time to evaluate the performance of the self-driving taxi agent. It can be evaluated according to the following metrics:
- **average number of steps per episode**: the smaller the number, the shorter the path taken by the agent to reach the destination;
- **average total reward per episode**: a higher average reward means that the agent reaches the destination as fast as possible with the least penalties;
- **average number of penalties per episode**: it is computed as the average number of time per episode the agent executes an illegal action (pick up or put down the passenger in the wrong location). The smaller the number, the better the performance of our agent.

By running the following code the evaluation metrics will be computed as the average of the results on several (*test_episode_count*) episodes.

In [ ]:
test_episode_count = 1000     # Total number of test episodes

sum_steps = 0
sum_total_reward = 0
sum_penalties = 0

for episode in range(test_episode_count):
    state = env.reset()
    for step in range(episode_max_steps):
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(Q[state])
        new_state, reward, done, _ = env.step(action)
        
        sum_steps += 1
        sum_total_reward += reward
        if reward == -10:
          sum_penalties += 1

        if done:
          break
        state = new_state

print ('Average number of steps per episode: {:.1f}'.format(sum_steps / test_episode_count))
print ('Average total reward per episode: {:.1f}'.format(sum_total_reward / test_episode_count))
print ('Average number of penalties per episode: {:.1f}'.format(sum_penalties / test_episode_count))

## **Single episode visualization**
The following code shows the self-driving in action on a single episode using the **print_taxi_step** function.

In [ ]:
episode_total_reward = 0

state = env.reset()
done = False
step = 0
while not done:
    action = np.argmax(Q[state])
    new_state, reward, done, _ = env.step(action)
    
    episode_total_reward += reward
    clear_output(wait = True)
    print_taxi_step(env.render(mode = 'ansi'), None, step, new_state, action, reward, episode_total_reward)
    time.sleep(0.5)
    
    state = new_state
    step += 1

# **Deep Q-learning**
Deep Q-Learning (DQL) is a variant of Q-Learning that uses a neural network as a function approximator to estimate the optimal action-value function, Q(s,a). In contrast to traditional Q-Learning, which stores Q-values in a table, DQL can handle high-dimensional state spaces, which makes it suitable for solving complex problems such as playing Atari games.

In DQL, the neural network takes the state as input and outputs the estimated Q-values for all possible actions. The network is trained using a variant of the Q-Learning update rule, where the target Q-value is computed using a fixed target network to prevent oscillations and instability:

y = r + γ max_a' Q'(s',a')

where r is the reward, γ is the discount factor, s' is the next state, a' is the action that maximizes the Q-value in the next state, and Q' is the fixed target network. The loss function for training the network is then given by the mean squared error between the predicted and target Q-values:

L = (y - Q(s,a))^2

The network is trained using stochastic gradient descent to minimize the loss function. The weights of the target network are periodically updated to match the weights of the online network.



# **Deep Q-network to solve the CartPole environment**
In this section, a Deep Q-Network (DQN) model is trained to solve the [**CartPole**](https://www.gymlibrary.ml/environments/classic_control/cart_pole/) environment provided by Gym.

## **CartPole environment**
The environment is represented by a pole attached to a cart which moves along a frictionless track. The objective is to prevent pole from falling over. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves far from the center.

### **Creation**
The following code creates the CartPole environment.

In [ ]:
env = gym.make('CartPole-v1')

### **Visualization**
To visualize the environment on *Colab*, [**PyVirtualDisplay**](https://pypi.org/project/PyVirtualDisplay/) is used to create a virtual display.

The following code creates a virtual display where the environment can be rendered. 

In [ ]:
display = Display()
display.start()

To visualize the current state of the environment, the **render** method can be used.

In [ ]:
env.reset()
prev_screen = env.render(mode = 'rgb_array')
plt.axis('off')
plt.imshow(prev_screen)
plt.show()

### **State space**
Differently from Taxi environment, the CartPole state space is represented by 4 continuous real values: 
- cart Position;
- cart Velocity;
- pole angle;
- pole angular velocity.

In this case, the **observation_space** attribute returns a [**Box**](https://www.gymlibrary.ml/content/api/#spaces) space class instance where valid states are in the form of *n*-dimensional arrays. The *shape* of the space can be obtained by the **shape** property.

In [ ]:
print('State space shape: ', env.observation_space.shape)

### **Action space**
The action space is composed by two possible actions:
- push cart to the left;
- push cart to the right.

The following code uses the **n** property of the action space to print the number of valid actions.

In [ ]:
print('There are {} possible actions'.format(env.action_space.n))

### **Rewards**
Reward is 1 for every step taken, including the termination step.

## **Model definition**
The following function creates a simple neural network to be used as DQN given:
- the number of input features (*input_count*);
- the number of neurons for each hidden layer (*neuron_count_per_hidden_layer*);
- the number of valid actions (*action_count*).

In Keras, a sequential is a stack of layers where each layer has exactly one input and one output. It can be created by passing a list of layers to the  constructor [**keras.Sequential**](https://keras.io/guides/sequential_model/).

[**Keras layers API**](https://keras.io/api/layers/) offers a wide range of built-in layers ready for use, including:
- [**Input**](https://keras.io/api/layers/core_layers/input/) - the input of the model. Note that, you can also omit the **Input** layer. In that case the model doesn't have any weights until the first call to a training/evaluation method (since it is not yet built);
- [**Dense**](https://keras.io/api/layers/core_layers/dense/) - a fully-connected layer.

In [ ]:
def build_simple_dqn(input_count, neuron_count_per_hidden_layer, action_count):
    model = keras.Sequential()
    model.add(layers.Input(shape = input_count, name = 'Input'))

    for n in neuron_count_per_hidden_layer:
        model.add(layers.Dense(n, activation = 'relu'))

    model.add(layers.Dense(action_count, name = 'Output'))

    return model

## **Action and target models creation**
To solve the *moving target problem*, two models (*action* and *target*) are used during the DQN training process.

The following code creates two identical models by calling the **build_simple_dqn** function defined above. To be identical, the two models need to share initial random weights: [**get_weights**](https://keras.io/api/models/model_saving_apis/#getweights-method) and [**set_weights**](https://keras.io/api/models/model_saving_apis/#setweights-method) methods are used to replace target weights to those of the action model.

In [ ]:
neuron_count_per_hidden_layer=[64, 32]

simple_dqn_action_model = build_simple_dqn(env.observation_space.shape, neuron_count_per_hidden_layer, env.action_space.n)
simple_dqn_target_model = build_simple_dqn(env.observation_space.shape, neuron_count_per_hidden_layer, env.action_space.n)
simple_dqn_target_model.set_weights(simple_dqn_action_model.get_weights())

## **Model visualization**
A string summary of the network can be printed using the [**summary**](https://keras.io/api/models/model/#summary-method) method.

In [ ]:
simple_dqn_action_model.summary()

The summary is useful for simple models, but can be confusing for complex models.

Function [**keras.utils.plot_model**](https://keras.io/api/utils/model_plotting_utils/) creates a plot of the neural network graph that can make more complex models easier to understand.

In [ ]:
keras.utils.plot_model(simple_dqn_action_model, show_shapes = True, show_layer_names = False)

## **Action model compilation**
The compilation is the final step in configuring the model for training. 

The following code uses the [**compile**](https://keras.io/api/models/model_training_apis/#compile-method) method to compile the model.
The important arguments are:
- the optimization algorithm (*optimizer*);
- the loss function (*loss*).

The most common [optimization algorithms](https://keras.io/api/optimizers/#available-optimizers) and [loss functions](https://keras.io/api/losses/#available-losses) are already available in Keras. You can either pass them to **compile** as an instance or by the corresponding string identifier. In the latter case, the default parameters will be used.

<u>Note that, only the action model is compiled because, during the training process, only its weights are updated using *gradient descent* algorithm. Target model weights are periodically replaced by those of the action model.</u>

In [ ]:
simple_dqn_action_model.compile(optimizer = 'adam', loss = 'mse')

## **Experience replay**
To remove correlations between consecutive transitions and make the DQN training more stable, the *experience replay* technique is used.

The replay memory is implemented as a [**deque**](https://docs.python.org/3/library/collections.html#collections.deque) (Doubly Ended Queue) object providing an O(1) time complexity for append and pop operations from both the ends of the queue. Moreover, if the *maxlen* parameter is specified, the **deque** is bounded to the specified maximum length. Once a bounded length deque is full, when new items are added, a corresponding number of items are discarded from the opposite end.

The DQN authors suggest to populate the replay memory before starting the learning process. For each step $t$, a random action is chosen and executed then the transition $<s_t,a_t,r_{t+1},s_{t+1}>$ is stored in the replay memory. The following function initializes the replay memory given:
- the environment (*env*);
- the replay memory (*replay_memory*);
- the number of transitions to be stored in the replay memory (*replay_memory_init_size*);
- the maximum number of steps per episode (*episode_max_steps*). 


In [ ]:
def simple_dqn_replay_memory_init(env, replay_memory, replay_memory_init_size, episode_max_steps):
    while True:
        state = env.reset()
        done = False
        step_count = 0
        while (step_count < episode_max_steps) and (not done):
            action = env.action_space.sample()
            new_state, reward, done, _ = env.step(action)
            replay_memory.append([state, action, reward, new_state, done])
            state = new_state
            step_count += 1
            if len(replay_memory) >= replay_memory_init_size:
                return

During training, when the action model needs to be updated, a mini-batch is randomly sampled from the replay memory and used instead of the most recent transition. The following function returns a mini-batch containing transitions randomly chosen from the replay memory given:
- the replay memory (*replay_memory*);
- the mini-batch size (*batch_size*).

In [ ]:
def get_random_batch_from_replay_memory(replay_memory, batch_size):
    minibatch_indices = np.random.choice(range(len(replay_memory)), size = batch_size)
    minibatch = [replay_memory[i] for i in minibatch_indices]
    
    state_batch = np.array([sample[0] for sample in minibatch])
    action_batch = np.array([sample[1] for sample in minibatch])
    reward_batch = np.array([sample[2] for sample in minibatch])
    new_state_batch = np.array([sample[3] for sample in minibatch])
    done_batch = np.array([sample[4] for sample in minibatch])

    return [state_batch, action_batch, reward_batch, new_state_batch, done_batch]

## **Exercise 2: DQN training algorithm**
Implement the following function to train a DQN model given:
- the environment (*env*);
- the action model (*dqn_action_model*);
- the target model (*dqn_target_model*);
- the number of episodes (*episode_count*);
- the maximum number of steps per episode (*episode_max_steps*);
- the maximum number of transitions stored into the replay memory (*replay_memory_max_size*);
- the number of transitions stored into the replay memory before starting the training process (*replay_memory_init_size*);
- the mini-batch size (*batch_size*);
- the number ($u$) of total steps executed ($T$) between successive updates of the action model weights (*step_per_update*);
- the number ($c$) of total steps executed ($T$) between successive replaces of the target model weights with the weights of the action model (*step_per_update_target_model*); 
- the maximum value of $\epsilon$ (*max_epsilon*);
- the minimum value of $\epsilon$ (*min_epsilon*);
- the $\epsilon$ decay value (*epsilon_decay*);
- the discount factor $\gamma$ (gamma);
- the number of consecutive episodes to be considered in the calculation of the total reward moving average (*moving_avg_window_size*);
- the minimum value of the total reward moving average to consider the task solved (*moving_avg_stop_thr*). A value of *None* force to execute all *episode_count* episodes.

It returns a **List** (*train_rewards*) containing the total rewards of all training episodes.

The following image shows the pseudo-code of the DQN training algorithm.

<img src=https://raw.githubusercontent.com/nderus/Generative-models-Lesson/main/images/DQN_training_algorithm.png width="800">

The following parts need to be implemented:
1. call the **simple_dqn_replay_memory_init** function to populate the replay memory before starting the training process;
2. initialize $\epsilon$ equal to *max_epsilon*;
3. call the **reset** method provided by the Gym environment interface to reset the environment and get the random initial state ($s_0$);
4. decide whether to pick a random action or to exploit the already computed Q-values. To this purpose, the **uniform** function provided by the random module can be used to generate a random number in the range $[0;1]$ to be compared against $\epsilon$. If a random action needs to be selected from the set of all possible actions, the **action_space.sample** method provided by the Gym environment can be exploited. Otherwise, use the [**predict**](https://keras.io/api/models/model_training_apis/#predict-method) method of the action model to obtain the Q-values of each available actions given the current state ($s_t$) and the  [**argmax**](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) function provided by Numpy to select the optimal one;
5. call the **step** method provided by the Gym environment interface to
execute the action $a_t$ selected at the previous point. It returns the new observation (or state) $s_{t+1}$, the reward $r_{t+1}$ achieved by action $a_t$, a boolean flag indicating if the episode has terminated (*True*) or not (*False*) and other additional information;
6. call the **get_random_batch_from_replay_memory** function to get a mini-batch of randomly selected transitions from the replay memory;
7. copy weights from action to target model using the **get_weights** and **set_weights** methods;
8. reduce $\epsilon$ at each step.

In [ ]:
def simple_dqn_training(env, dqn_action_model, dqn_target_model, episode_count, episode_max_steps,
                        replay_memory_max_size, replay_memory_init_size, batch_size,
                        step_per_update, step_per_update_target_model,
                        max_epsilon, min_epsilon, epsilon_decay, gamma,
                        moving_avg_window_size = 20, moving_avg_stop_thr = None):
    # experience replay memory declaration
    replay_memory = deque(maxlen = replay_memory_max_size)
    
    # 1. replay memory initial population
    if replay_memory_init_size > 0:
        print('Replay memory initial population')
        simple_dqn_replay_memory_init(env, replay_memory, replay_memory_init_size, episode_max_steps)

    # 2. inizialize epsilon equal to max_epsilon
    epsilon = max_epsilon

    train_rewards = []  # for visualization purposes

    train_step_count = 0  #T
    for n in range(episode_count): 
        # for visualization purposes
        episode_start_time = time.time()    
        episode_reward = 0
        episode_epsilon = epsilon

        # 3. reset the environment
        state = env.reset()

        step_count = 0  #t
        done = False
        while step_count < episode_max_steps and not done: 
            # 4. decide whether to pick a random action or to exploit the already computed Q-values
            if random.uniform(0, 1) <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = dqn_action_model.predict(state[np.newaxis])
                action = np.argmax(q_values)
            
            # 5. take the action and observe the outcome state and reward
            new_state, reward, done, _ = env.step(action)
            
            # store transition in the replay memory 
            replay_memory.append([state, action, reward, new_state, done])
            
            # update the current state
            state = new_state
                        
            if train_step_count % step_per_update == 0 and len(replay_memory) >= batch_size:
                # 6. get a random mini-batch from the replay memory
                mini_batch = get_random_batch_from_replay_memory(replay_memory, batch_size)
                
                # update the action model weights calling the simple_dqn_update function (see below)
                dqn_action_model = simple_dqn_update(dqn_action_model, dqn_target_model, mini_batch, gamma)
                
            if train_step_count % step_per_update_target_model == 0:
                # 7. copy weights from action to target model
                dqn_target_model.set_weights(dqn_action_model.get_weights())

            # 8. reduce epsilon
            if epsilon > min_epsilon:
                epsilon = max(min_epsilon, epsilon - epsilon_decay)

            # increase episode step count and total step count        
            step_count += 1
            train_step_count += 1

            # add the current reward to the episode total reward
            episode_reward += reward 
        
        # put the episode total reward into a list for visualization purposes
        train_rewards.append(episode_reward)

        # for visualization purposes
        episode_finish_time = time.time()
        episode_elapsed_time = episode_finish_time - episode_start_time
        episode_avg_step_time = episode_elapsed_time / step_count
        moving_avg_reward = statistics.mean(train_rewards[- moving_avg_window_size:])
        print("Episode: {} Steps: {}[{}] Epsilon: {:.3f} Time: {:.1f}s[{:.2f}s]  Total reward: {}[{:.1f}]".format(n,
                                                                                                                  step_count,
                                                                                                                  train_step_count,
                                                                                                                  episode_epsilon,
                                                                                                                  episode_elapsed_time,
                                                                                                                  episode_avg_step_time,
                                                                                                                  episode_reward,
                                                                                                                  moving_avg_reward))
        
        # condition to consider the task solved
        if (moving_avg_stop_thr is not None) and moving_avg_reward>moving_avg_stop_thr:
            break

    # return a list containing the total rewards of all training episodes  
    return train_rewards

### **Action model update**
Implement the **simple_dqn_update** function to update action model weights using *gradient descent* algorithm given:
- the action model (*dqn_action_model*);
- the target model (*dqn_target_model*);
- a mini-batch containing transitions $<s_i,a_i,r_{i+1},s_{i+1}>$ randomly selected from the replay memory (*mini_batch*); 
- the discount factor $\gamma$ (gamma).

The following parts need to be implemented:
1. estimate $\hat{Q}(s_{i+1},a)$ for all possible actions using the **predict** method of the target model with *new_state_batch* ($s_{i+1}$) as input parameter;
2. estimate $Q(s_i,a)$ for all possible actions using the **predict** method of the action model with *state_batch* ($s_i$) as input parameter;
3. update weights of the action model using the [**train_on_batch**](https://keras.io/api/models/model_training_apis/#trainonbatch-method) method with *state_batch* ($s_i$) and *predicted_state_q_values* ($y_i$) as *x* and *y* input parameters, respectively.

In [ ]:
def simple_dqn_update(dqn_action_model, dqn_target_model, mini_batch, gamma):
    # the transition mini-batch is divided into a mini-batch for each element of a transition
    state_batch,action_batch, reward_batch, new_state_batch, done_batch = mini_batch

    # 1. find the target model Q values for all possible actions given the new state batch
    target_new_state_q_values = dqn_target_model.predict(new_state_batch)
    
    # 2. find the action model Q values for all possible actions given the current state batch
    predicted_state_q_values = dqn_action_model.predict(state_batch)
    
    # estimate the target values y_i
    # for the action we took, use the target model Q values  
    # for other actions, use the action model Q values
    # in this way, loss function will be 0 for other actions
    for i, (a, r, new_state_q_values, done) in enumerate(zip(action_batch, reward_batch, target_new_state_q_values, done_batch)): 
        if not done:  
          target_value = r + gamma * np.amax(new_state_q_values)
        else:         
          target_value = r
        predicted_state_q_values[i][a] = target_value #y_i
    
    # 3. update weights of action model using the train_on_batch method 
    dqn_action_model.train_on_batch(state_batch, predicted_state_q_values)
    
    # return the updated action model
    return dqn_action_model

## **Training**
Now we are ready to start the training process by calling the **simple dqn_learning** function.

The following parameters must be set before being passed to the function:
- the number of episodes (*episode_count*);
- the maximum number of steps per episode (*episode_max_steps*);
- the maximum number of transitions stored into the replay memory (*replay_memory_max_size*);
- the number of transitions stored into the replay memory before starting the training process (*replay_memory_init_size*);
- the mini-batch size (*batch_size*);
- the number of total steps executed between successive updates of the action model weights (*step_per_update*);
- the number of total steps executed between successive replaces of the target model weights with the weights of the action model (*step_per_update_target_model*); 
- the maximum value of $\epsilon$ (*max_epsilon*);
- the minimum value of $\epsilon$ (*min_epsilon*);
- the $\epsilon$ decay value (*epsilon_decay*);
- the discount factor $\gamma$ (gamma);
- the number of consecutive episodes to be considered in the calculation of the total reward moving average (*moving_avg_window_size*);
- the minimum value of the total reward moving average to consider the task solved (*moving_avg_stop_thr*).

In [ ]:
episode_count = 500               # Total number of training episodes
episode_max_steps = 400             # Maximum number of steps per episode

replay_memory_max_size = 100000   # Maximum number of transitions stored into the replay memory
replay_memory_init_size = 1000      # Maximum number of transitions stored into the replay memory
batch_size = 64                   # Mini-batch size

step_per_update = 4               # Number of total steps executed between successive updates of the action model weights
step_per_update_target_model = 8    # Number of total steps executed between successive replaces of the target model weights

max_epsilon = 1.0                   # Exploration probability at start
min_epsilon = 0.01                  # Minimum exploration probability
epsilon_decay = 0.0002              # Decay for exploration probability

gamma = 0.99                      # Discount factor

moving_avg_window_size = 20         # Number of consecutive episodes to be considered in the calculation of the total reward moving average
moving_avg_stop_thr = 100           # Minimum value of the total reward moving average to consider the task solved

train_start_time = time.time()
train_rewards=simple_dqn_training(env,
                                  simple_dqn_action_model,
                                  simple_dqn_target_model,
                                  episode_count,
                                  episode_max_steps,
                                  replay_memory_max_size,
                                  replay_memory_init_size,
                                  batch_size,
                                  step_per_update,
                                  step_per_update_target_model,
                                  max_epsilon,
                                  min_epsilon,
                                  epsilon_decay,
                                  gamma,
                                  moving_avg_window_size,
                                  moving_avg_stop_thr)

train_finish_time = time.time()
train_elapsed_time = train_finish_time - train_start_time
train_avg_episode_time = train_elapsed_time / episode_count
print("Train time: {:.1f}m [{:.1f}s]".format(train_elapsed_time / 60.0, train_avg_episode_time))

### **Visualize the training process**
It is important to observe how performance changes over time during the training process.

The **plot_training_rewards** function defined above is used to draw in a graph the total reward and its moving average reached during the different episodes of the training process.

In [ ]:
plot_training_rewards(train_rewards, moving_avg_window_size)

## **Performance evaluation**
The CartPole environment can be considered solved when the average total reward over 100 consecutive trials is greater than or equal to 195.

In [ ]:
episode_count = 5

sum_episode_rewards = 0
for episode in range(episode_count):
  state = env.reset()

  episode_reward = 0
  done = False
  step_count = 0
  while not done:
      q_values = simple_dqn_action_model.predict(state[np.newaxis])
      action = np.argmax(q_values)
      
      new_state, reward, done, _ = env.step(action)

      state = new_state
      episode_reward += reward

  sum_episode_rewards += episode_reward

  print('Episode: {} Total reward: {}'.format(episode, episode_reward))

print('Average total reward: {:.1f}'.format(sum_episode_rewards / episode_count))

## **Single episode visualization**
To show the CartPole agent in action on *Colab*, it is not possible to render each frame at real time.

It is necessary to:
1. execute an entire episode storing all episode frames;
2. create an MP4 video with all frames;
3. visualize the video.

The following code executes an episode using the trained DQN network and store all frames into a list (*frames*).

In [ ]:
state = env.reset()

frames = []
done = False
episode_reward = 0
while not done:
    frame = env.render(mode = 'rgb_array')
    frames.append(frame)
    
    q_values = simple_dqn_action_model.predict(state[np.newaxis])
    action = np.argmax(q_values)
    
    new_state, reward, done, _ = env.step(action)
    state = new_state

    episode_reward += reward

print('Total reward: ', episode_reward)

Then the MP4 video file, created using the **create_mp4_video_from_frames** function, is included into an HTML video tag.

In [ ]:
vide_file_name = create_mp4_video_from_frames(frames, 30)

mp4 = open(vide_file_name, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(""" <video controls><source src="%s" type="video/mp4"></video>""" % data_url)